In [10]:
#import dependencies
import os
import json
from dotenv import load_dotenv
from supabase import create_client, Client
import requests
import pandas as pd

In [9]:
#Initialize supabase connection

load_dotenv()
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")
SUPABASE_URL = os.getenv("SUPABASE_URL")
GOOGLE_GEOCODE_KEY = os.getenv('GOOGLE_GEOCODE_API')

supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
print("Supabase client initialized successfully!")

Supabase client initialized successfully!


In [16]:
#Helper function to read dataset
def getData(filePath):
    with open(filePath, 'r') as f:
        data = json.load(f)
        return data

In [17]:
#Helper function to get geocode with google API

def get_geocode(address_data):
    address = f"{address_data['premise']} {address_data['street']}, {address_data['locality']}, {address_data['administrative_area']}"
    uri = f"https://maps.googleapis.com/maps/api/geocode/json?key={GOOGLE_GEOCODE_KEY}&address={address}"
    response = requests.get(uri)
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            location = data['results'][0]['geometry']['location']
            return location['lat'], location['lng']
    return None, None

In [ ]:
#Insert function into supabase
def insert_data(data):
    for entry in data:
        address_data = {
            'country': entry['address']['country'] or '',
            'administrative_area': entry['address']['administrative_area'] or '',
            'sub_administrative_area': entry['address']['sub_administrative_area'] or '',
            'locality': entry['address']['locality'] or '',
            'postal_code': entry['address']['postal_code'] or '',
            'street': entry['address']['street'] or '',
            'premise': entry['address']['premise'] or '',
            'sub_premise': entry['address']['sub_premise'] or '',
        }

        # geocode for the address
        lat, lng = get_geocode(address_data)
        address_data['latitude'] = lat
        address_data['longitude'] = lng

        address_response = supabase.table('address').upsert(
            [address_data],
            on_conflict="country,administrative_area,sub_administrative_area,locality,postal_code,street,premise,sub_premise"
        ).execute()
        
        if address_response.data:
            address_id = address_response.data[0]['id']
        else:
            print(f"Error upserting address data: {address_response}")
            continue 
        
        # Convert None to 0 for numeric entries
        price = entry['price'] if entry['price'] is not None else 0
        bedrooms = entry['bedrooms'] if entry['bedrooms'] is not None else 0
        bathrooms = entry['bathrooms'] if entry['bathrooms'] is not None else 0
        square_feet = entry['square_feet'] if entry['square_feet'] is not None else 0
        acre_lot = entry['acre_lot'] if entry['acre_lot'] is not None else 0
        
        property_listing_data = {
            'price': price,
            'bedrooms': bedrooms,
            'bathrooms': bathrooms,
            'square_feet': square_feet,
            'sale_status': entry['sale_status'],
            'acre_lot': acre_lot,
            'tour_available': entry['tour_available'],
            'image_source': entry['image_source'],
            'address_id': address_id
        }
        
        property_response = supabase.table('property_listings').upsert(
            [property_listing_data],
            on_conflict='address_id,price,sale_status'
        ).execute()
        
        if property_response.data:
            print(f"Successfully upserted property listing: {property_listing_data}")
        else:
            print(f"Error upserting property listing data: {property_response}")

In [20]:
#load json files
brooklyn_data = getData('../scraper/json-dump/brooklyn-2025-03-15-formatted.json')
manhattan_data = getData('../scraper/json-dump/manhattan-2025-03-15-formatted.json')
queens_data = getData('../scraper/json-dump/queens-2025-03-15-formatted.json')
staten_island_data = getData('../scraper/json-dump/staten-island-2025-03-15-formatted.json')
bronx_data = getData('../scraper/json-dump/bronx-2025-03-15-formatted.json')

In [23]:
insert_data(bronx_data)

Successfully upserted property listing: {'price': 700000, 'bedrooms': 4, 'bathrooms': 2.5, 'square_feet': 1863.0, 'sale_status': 'House for sale', 'acre_lot': 1800.0, 'tour_available': True, 'image_source': 'https://ap.rdcpix.com/ffa67aa2e123357cf879c2b693022beal-m2482881542rd-w960_h720.jpg', 'address_id': 1}
Successfully upserted property listing: {'price': 699999, 'bedrooms': 4, 'bathrooms': 2.5, 'square_feet': 1260.0, 'sale_status': 'House for sale', 'acre_lot': 1800.0, 'tour_available': True, 'image_source': 'https://ap.rdcpix.com/2dca2e942e667f7368c974057877d8bel-m2008829264rd-w960_h720.jpg', 'address_id': 2}
Successfully upserted property listing: {'price': 849000, 'bedrooms': 8, 'bathrooms': 3.0, 'square_feet': 2150.0, 'sale_status': 'Townhouse for sale', 'acre_lot': 3257.0, 'tour_available': True, 'image_source': 'https://ap.rdcpix.com/8b41918b1ee6b36f10c4e90e2edccd0dl-m1123716120rd-w960_h720.jpg', 'address_id': 3}


KeyboardInterrupt: 

In [22]:
insert_data(manhattan_data)

APIError: {'code': '42P10', 'details': None, 'hint': None, 'message': 'there is no unique or exclusion constraint matching the ON CONFLICT specification'}

In [ ]:
insert_data(brooklyn_data)

In [ ]:
insert_data(queens_data)

In [ ]:
insert_data(staten_island_data)

In [60]:
pd.set_option('display.max_rows', 10)

In [67]:
def fetch_all_data(table_name, columns):
    """Fetch all rows from a Supabase table with pagination."""
    all_data = []
    chunk_size = 1000
    start = 0

    while True:
        response = supabase.table(table_name).select(columns).range(start, start + chunk_size - 1).execute()
        if not response.data:
            break
        all_data.extend(response.data)
        start += chunk_size

    return all_data


def fetch_address_and_property_data():
    # Fetch all address data
    address_data = fetch_all_data(
        'address',
        'id, postal_code, locality, administrative_area, sub_administrative_area'
    )
    if not address_data:
        print("No address data found.")
        return pd.DataFrame()

    # Fetch all property listing data
    property_data = fetch_all_data(
        'property_listings',
        'address_id, price, square_feet, sale_status'
    )
    if not property_data:
        print("No property listing data found.")
        return pd.DataFrame()

    # Merge address and property data
    address_df = pd.DataFrame(address_data)
    property_df = pd.DataFrame(property_data)

    merged_df = pd.merge(address_df, property_df, left_on='id', right_on='address_id', how='inner')

    # Drop redundant columns
    merged_df = merged_df.drop(columns=['address_id'])

    # Calculate price per square foot
    merged_df['price_per_sqft'] = merged_df.apply(
        lambda row: row['price'] / row['square_feet'] if row['square_feet'] > 0 else None, axis=1
    )

    # Calculate medians for fallback
    zip_median = merged_df.groupby('postal_code')['price_per_sqft'].median()
    locality_median = merged_df.groupby('locality')['price_per_sqft'].median()
    sub_admin_median = merged_df.groupby('sub_administrative_area')['price_per_sqft'].median()
    global_median = merged_df['price_per_sqft'].median()

    # Fill missing price_per_sqft with fallback logic
    def fill_price_per_sqft(row):
        if pd.notnull(row['price_per_sqft']):
            return row['price_per_sqft']
        if pd.notnull(zip_median.get(row['postal_code'])):
            return zip_median[row['postal_code']]
        if pd.notnull(locality_median.get(row['locality'])):
            return locality_median[row['locality']]
        if pd.notnull(sub_admin_median.get(row['sub_administrative_area'])):
            return sub_admin_median[row['sub_administrative_area']]
        return global_median

    merged_df['price_per_sqft'] = merged_df.apply(fill_price_per_sqft, axis=1)

    return merged_df

# Fetch and display the data
result_df = fetch_address_and_property_data()
result_df = result_df.sort_values(by='id', ascending=True)

# Print the DataFrame in a readable format
result_df

,id,postal_code,locality,administrative_area,sub_administrative_area,price,square_feet,sale_status,price_per_sqft
6931,1,10469,Bronx,NY,,700000,1863.0,House for sale,375.738057
2829,2,10469,Bronx,NY,,699999,1890.0,House for sale,370.369841
6815,3,10466,Bronx,NY,,849000,2150.0,Townhouse for sale,394.883721
6771,4,10462,Bronx,NY,,290000,900.0,Condo for sale,322.222222
2840,5,10473,Bronx,NY,,579000,1311.0,Condo for sale,441.647597
...,...,...,...,...,...,...,...,...,...
27817,76933,10304,Staten Island,NY,Staten Island,2299000,6600.0,House for sale,348.333333
27842,77000,10304,Staten Island,NY,Staten Island,1600000,3400.0,Multi-family home for sale,470.588235
27843,77003,10301,Staten Island,NY,Staten Island,699999,1260.0,Townhouse for sale,555.554762
27882,77084,10308,Staten Island,NY,Staten Island,688000,1486.0,House for sale,462.987887


In [72]:
from statsmodels import robust

# Precompute MADs for fallback logic
zip_mads = result_df.groupby("postal_code")["price_per_sqft"].apply(robust.mad).to_dict()
locality_mads = result_df.groupby("locality")["price_per_sqft"].apply(robust.mad).to_dict()
sub_area_mads = result_df.groupby("sub_administrative_area")["price_per_sqft"].apply(robust.mad).to_dict()
overall_mad = robust.mad(result_df["price_per_sqft"])

# Precompute medians for fallback logic
zip_medians = result_df.groupby("postal_code")["price_per_sqft"].median().to_dict()
locality_medians = result_df.groupby("locality")["price_per_sqft"].median().to_dict()
sub_area_medians = result_df.groupby("sub_administrative_area")["price_per_sqft"].median().to_dict()
overall_median = result_df["price_per_sqft"].median()

# Function to score fairness using MAD
def score_fairness_mad(row):
    zip_code = row["postal_code"]
    locality = row["locality"]
    sub_area = row["sub_administrative_area"]
    ppsf = row["price_per_sqft"]

    # Median price per sqft (fallback logic)
    median_ppsf = zip_medians.get(zip_code)
    if pd.isna(median_ppsf):
        median_ppsf = locality_medians.get(locality)
    if pd.isna(median_ppsf):
        median_ppsf = sub_area_medians.get(sub_area)
    if pd.isna(median_ppsf):
        median_ppsf = overall_median

    # MAD per sqft (fallback logic)
    mad_ppsf = zip_mads.get(zip_code)
    if mad_ppsf is None or mad_ppsf == 0:
        mad_ppsf = locality_mads.get(locality)
    if mad_ppsf is None or mad_ppsf == 0:
        mad_ppsf = sub_area_mads.get(sub_area)
    if mad_ppsf is None or mad_ppsf == 0:
        mad_ppsf = overall_mad

    # Avoid division by zero or NaN
    if mad_ppsf == 0 or pd.isna(ppsf) or pd.isna(median_ppsf):
        return "Unknown"

    # Modified z-score using MAD
    modified_z = 0.6745 * (ppsf - median_ppsf) / mad_ppsf

    # Thresholding on modified z-score
    if modified_z < -0.4:
        return "good"
    elif -0.4 <= modified_z <= 0.4:
        return "fair"
    else:
        return "bad"

# Apply the function to the DataFrame
result_df["fairness_rating"] = result_df.apply(score_fairness_mad, axis=1)

fairness_df = result_df.rename(columns={"id": "address_id"})[["address_id", "price", "sale_status", "fairness_rating"]]
fairness_df

,address_id,price,sale_status,fairness_rating
6931,1,700000,House for sale,good
2829,2,699999,House for sale,good
6815,3,849000,Townhouse for sale,fair
6771,4,290000,Condo for sale,good
2840,5,579000,Condo for sale,bad
...,...,...,...,...
27817,76933,2299000,House for sale,good
27842,77000,1600000,Multi-family home for sale,fair
27843,77003,699999,Townhouse for sale,bad
27882,77084,688000,House for sale,fair


In [75]:
def update_fairness_rating(batch_size=100):
    # Split the DataFrame into smaller batches
    for start in range(0, len(fairness_df), batch_size):
        batch = fairness_df.iloc[start:start + batch_size]
        updates = []

        for _, row in batch.iterrows():
            updates.append({
                'address_id': int(row['address_id']),  # Convert to native Python int
                'fairness_rating': row['fairness_rating'],
                'price': int(result_df.loc[result_df['id'] == row['address_id'], 'price'].values[0]),  # Convert to native Python int
                'sale_status': result_df.loc[result_df['id'] == row['address_id'], 'sale_status'].values[0]  # Ensure sale_status is included
            })

        # Perform batch updates
        response = supabase.table('property_listings').upsert(updates, on_conflict='address_id').execute()

        if response.data is not None:
            print(f"Successfully updated fairness_rating for batch starting at index {start}")
        else:
            print(f"Failed to update fairness_rating for batch starting at index {start}: {response.errors}")

update_fairness_rating()

Successfully updated fairness_rating for batch starting at index 0
Successfully updated fairness_rating for batch starting at index 100
Successfully updated fairness_rating for batch starting at index 200
Successfully updated fairness_rating for batch starting at index 300
Successfully updated fairness_rating for batch starting at index 400
Successfully updated fairness_rating for batch starting at index 500
Successfully updated fairness_rating for batch starting at index 600
Successfully updated fairness_rating for batch starting at index 700
Successfully updated fairness_rating for batch starting at index 800
Successfully updated fairness_rating for batch starting at index 900
Successfully updated fairness_rating for batch starting at index 1000
Successfully updated fairness_rating for batch starting at index 1100
Successfully updated fairness_rating for batch starting at index 1200
Successfully updated fairness_rating for batch starting at index 1300
Successfully updated fairness_ra